## Computer Vision PA #2: Structure from Motion (SfM)
20195003 고강빈

In [1]:
import numpy as np
import cv2 as cv

In [5]:
img1 = cv.imread('./SfM/Data/sfm03.jpg')
img2 = cv.imread('./SfM/Data/sfm04.jpg')

# create SIFT instance
sift = cv.SIFT_create()
# sift = cv.xfeatures2d.SIFT_create()

# detect and compute keypoints
img1_kp, img1_des = sift.detectAndCompute(img1, None)
img2_kp, img2_des = sift.detectAndCompute(img2, None)

img1_drawKps = cv.drawKeypoints(img1, img1_kp, None)
img2_drawKps = cv.drawKeypoints(img2, img2_kp, None)

# save result
cv.imwrite('sift_keypoints.jpg',img1_drawKps)

print(img1_des.shape)
print(img1_des)

(37936, 128)
[[  1.   0.   0. ...   1.   7.   7.]
 [  3.   1.   2. ...   3.   1.   1.]
 [  0.   1.   2. ...   0.   0.   0.]
 ...
 [  1.   0.   0. ...   2.   0.   1.]
 [  0.   0.   0. ...  17.  11.  16.]
 [145.  10.   2. ...   0.   0.   0.]]


In [6]:
# Brute force matching with k=2
bf = cv.BFMatcher()
matches = bf.knnMatch(img1_des, img2_des, k=2)

# Ratio test and retrieval of indices
matches_good = [m1 for m1, m2 in matches if m1.distance < 0.75*m2.distance]

sorted_matches = sorted(matches_good, key=lambda x: x.distance)
res = cv.drawMatches(img1, img1_kp, img2, img2_kp, sorted_matches, img2, flags=2) 

cv.imwrite('sift_bfMatcher.jpg',res)
print(len(sorted_matches))

1893


In [7]:
# print(res.shape)
# print(res)

In [9]:
# queryIdx : 1번 이미지의 feature point index
# trainIdx : 2번 이미지의 feature point index
query_idx = [match.queryIdx for match in matches_good]
train_idx = [match.trainIdx for match in matches_good]

#Getting float based points from good matches
p1 = np.float32([img1_kp[ind].pt for ind in query_idx])
p2 = np.float32([img2_kp[ind].pt for ind in train_idx])

print(p1.shape)
print(p1[:3])

img1_pts = np.array([img1_kp[m.queryIdx].pt for m in matches_good]).reshape(-1, 1, 2).astype(np.float32) # 픽셀 좌표
img2_pts = np.array([img2_kp[m.trainIdx].pt for m in matches_good]).reshape(-1, 1, 2).astype(np.float32)

print(img1_pts.shape)
print(img1_pts[:3])

(1893, 2)
[[  24.099752 3219.431   ]
 [ 298.76065  2282.336   ]
 [ 330.07364  2516.3718  ]]
(1893, 1, 2)
[[[  24.099752 3219.431   ]]

 [[ 298.76065  2282.336   ]]

 [[ 330.07364  2516.3718  ]]]


In [18]:
p1

array([[  24.099752, 3219.431   ],
       [ 298.76065 , 2282.336   ],
       [ 330.07364 , 2516.3718  ],
       ...,
       [4551.96    , 3080.2961  ],
       [4573.4385  , 3094.0754  ],
       [4574.8506  , 2760.8474  ]], dtype=float32)

In [63]:
def randomsample(p1, p2):
    concat = np.concatenate((p1, p2), axis=1)
    rand_mat = concat[np.random.randint(concat.shape[0], size = len(concat)), :]
    p1s = rand_mat[:5, :2]
    p2s = rand_mat[:5, 2:]
    return p1s, p2s

In [59]:
concat = np.concatenate((p1, p2), axis=1)
print(concat.shape)
print(len(concat))

(1893, 4)
1893


In [11]:
# def RANSAC2(p1, p2, iter, threshold):
    
#     b_inlier = np.array([[]])
#     b_E = None
#     tmp_inlier_len = 0

#     for i in range(iter):
#     	# 5개의 Random sample 추출
#         p1s, p2s = randomsample(p1, p2)
        
#         # 5-point algorithm
#         E, cur_inlier = cv2.findEssentialMat(p1s, p2s, maxIters = 0)

#         # inlier 추출
#         inlier_idx = np.where(cur_inlier==1)
#         pts1 = np.vstack([p1[i] for i in inlier_idx[0]])
#         pts2 = np.vstack([p2[i] for i in inlier_idx[0]])
        
#         # intrinsic parameter 설정
#         skew = 0.0215878
#         K = np.array([[3092.8, skew, 2016], [0, 3092.8, 1512], [0,0,1]])
#         K_inv = np.linalg.inv(K)
        
#         # (x,y) -> Homogeneous Coordinate (x,y,1) 로 변환
#         a, b = rescale_point(pts1, pts2, len(pts1))

#         if E is None:
#             continue
        
#         # Epipolar constraint = 0
#         cur_E = (b @ K_inv @ E @ K_inv.T @ a.T) 
		
#         c = []
#         for i in range(len(pts1)):
#             c.append(cur_E[i][i])
#         c = np.array(c)
        
#         # inlier 수가 가장 많은 E 추출
#         tmp_inlier_len = len(c[(c<threshold) & (c>0)])
#         if len(pts1) > tmp_inlier_len: # 현재 inlier 수 > 최고 inlier 수
#             b_E = cur_E 
#             b_inlier = cur_inlier 
#             tmp_inlier_len = len(pts1)
    
#     return b_E

In [12]:
# # intrinsic parameter 
# Rt0 = np.hstack((np.eye(3), np.zeros((3, 1))))
# skew = 0.0215878
# K = np.array([[3092.8, skew, 2016], [0, 3092.8, 1512], [0,0,1]])
# Rt1 = K @ CameraMatrix

# # Rescale to Homogeneous Coordinate
# def rescale_point(pts1, pts2, length):
#     a = [[]]
#     b = [[]]
#     for i in range(length):
#         tmp1 = pts1[i].flatten()
#         tmp1 = np.append(tmp1, 1)
#         a = np.append(a, tmp1)
#         tmp2 = pts2[i].flatten()
#         tmp2 = np.append(tmp2, 1)
#         b = np.append(b, tmp2)
    
#     a = a.reshape((length),3)
#     b = b.reshape((length),3)
#     return a, b

# p1, p2 = rescale_point(p1, p2, len(p1))

# # Triangulation
# def LinearTriangulation(Rt0, Rt1, p1, p2):
#     A = [p1[1]*Rt0[2,:] - Rt0[1,:], # x(p 3row) - (p 1row) 
#         -(p1[0]*Rt0[2,:] - Rt0[0,:]), # y(p 3row) - (p 2row) 
#         p2[1]*Rt1[2,:] - Rt1[1,:], # x'(p' 3row) - (p' 1row) 
#         -(p2[0]*Rt1[2,:] - Rt1[0,:])]  # y'(p' 3row) - (p' 2row)
        
#     A = np.array(A).reshape((4,4))
#     AA = A.T @ A 
#     U, S, VT = np.linalg.svd(AA) # right singular vector
 
#     return VT[3,0:3]/VT[3,3]

# p3ds = []
# for pt1, pt2 in zip(p1, p2):
#     p3d = LinearTriangulation(Rt0, Rt1, pt1, pt2)
#     p3ds.append(p3d)
# p3ds = np.array(p3ds).T

In [66]:
import matlab.engine

MAXITER = 100
best_inlier = np.array([[]])
best_E = None
inlier_len_temp = 0

# intrinsic parameter
K = np.array([[3451.5, 0.0, 2312.0], [0.0, 3451.5, 1734], [0.0,0.0,1.0]])
K_inv = np.linalg.inv(K)

# for i in range(MAXITER):
    
# select random points
p1s, p2s = randomsample(p1, p2)
p1s, p2s = np.array(matlab.double(p1s.tolist())), np.array(matlab.double(p2s.tolist()))

# 5-points algorithm
# eng = matlab.engine.start_matlab()
# eng.addpath("./SfM/Step2/")  # 'calibrated_fivepoint.m'가 위치한 경로
# cur_E = np.asarray(eng.calibrated_fivepoint(p1s, p2s))
# eng.quit()

cur_E, cur_inlier = cv.findEssentialMat(p1s, p2s, focal=3451.5, pp=(2312, 1734), maxIters=0, threshold=0.1)

print(cur_E.shape)

if cur_E is None:
    print('Error: E is empty')

# estim = p1s * cur_E.T * p2s.T
# # inlier
# for i in range(cur_E.shape[1]):
#     tmp = cur_E[:,i]
#     tmp = np.reshape(tmp, [3,3])
#     for j in range(5):
#         estim = 




(6, 3)


In [17]:
# import matlab.engine
# eng = matlab.engine.start_matlab()
# eng.addpath("./SfM/Step2/")  # 'calibrated_fivepoint.m'가 위치한 경로
# for i in range(100):
#     a = np.random.rand(3, 5).tolist()
#     a = matlab.double(a)
#     b = np.random.rand(3, 5).tolist()
#     b = matlab.double(b)
#     E = eng.calibrated_fivepoint(a, b)
#     print(np.asarray(E))

In [47]:
def randomsample(p1, p2):
    p1p2 = np.concatenate((p1, p2), axis=1)
    p1p2_ = p1p2[np.random.randint(p1p2.shape[0], size=len(p1)), :]
    p1s = p1p2_[:,:2]
    p2s = p1p2_[:,2:]
    return p1s, p2s

    
b_inlier = np.array([[]]) # best inlier
b_E = None # best Essential Matrix
tmp_inlier_len = 0 # tmp inlier의 개수

# choice random sample
p1s, p2s = randomsample(p1, p2)
print(p1s.shape)

# 5-point algorithm (with intrinsic parameter, epipolar constraint = 0, no RANSAC) 
cur_E, cur_inlier = cv.findEssentialMat(p1s, p2s, focal=3092.8, pp=(2016, 1512),
                                            maxIters = 0, threshold=0.1)

print(cur_E)
print(cur_inlier)

(1893, 2)
[[ 0.04895976  0.38504826 -0.00336808]
 [-0.23916036  0.0168629  -0.66520118]
 [ 0.07145306  0.58656625 -0.01375776]]
[[0]
 [0]
 [0]
 ...
 [1]
 [0]
 [0]]
